In [3]:
import os
import logging
import yaml

from pyyoutube import Api
import json
import requests

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

import re
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import SpectralClustering
from sklearn.metrics import f1_score, silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.tracking import MlflowClient

[nltk_data] Downloading package punkt to /home/aegon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aegon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)

# handler = logging.FileHandler('../logging/logging.log')
# logger.addHandler(handler)

# formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# handler.setFormatter(formatter)

In [5]:
def get_data(YOUTUBE_API_KEY, video_id, max_results, next_page_token):
    youtube_uri = f'https://www.googleapis.com/youtube/v3/commentThreads?key={YOUTUBE_API_KEY}&textFormat=plainText&' + \
        f'part=snippet&videoId={video_id}&maxResults={max_results}&pageToken={next_page_token}'
    
    try: 
        response = requests.get(youtube_uri)
        response.raise_for_status()
        data = json.loads(response.text)
    except Exception as e:
        return None
    return data


def get_text_of_comment(data):
    comms = set()
    for item in data['items']:
        comm = item['snippet']['topLevelComment']['snippet']['textDisplay']
        comms.add(comm)
    return comms


def get_all_comments(YOUTUBE_API_KEY, query, count_video=10, limit=30, max_results=10, next_page_token=''):
    api = Api(api_key=YOUTUBE_API_KEY)
    video_by_keywords = api.search_by_keywords(q=query,
                                               search_type=["video"],
                                               count=count_video,
                                               limit=limit
    )
    video_ids = [x.id.videoId for x in video_by_keywords.items]

    comments_all = []
    for video_id in video_ids:
        try:
            data = get_data(YOUTUBE_API_KEY,
                            video_id=video_id,
                            max_results=max_results,
                            next_page_token=next_page_token
            )
            
            if 'items' in data:
                comment = list(get_text_of_comment(data))
                comments_all.append(comment)
        except Exception as e:
            continue

    comments = sum(comments_all, [])
    return comments


In [6]:
config_path = os.path.join('/home/aegon/Documents/Development/Projects/topic-sentiment-explorer/config/params_all.yaml')
config = yaml.safe_load(open(config_path))['train']
config['clustering']

{'affinity': 'cosine',
 'count_max_clusters': 15,
 'silhouette_metric': 'euclidean'}

In [7]:
SEED = config['SEED']

comments = get_all_comments(**config['comments'])
comments[:10]

In [ ]:
def remove_emoji(sentence):
    emoji_pattern = re.compile("["u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', sentence)


def remove_links(sentence):
    link_pattern = r'(http\S+|bit\.ly/\S+|www\S+)'
    sentence = re.sub(link_pattern, '', sentence)
    sentence = sentence.strip('[link]')
    return sentence


def preprocessing(sentence, stop_words, lemmatizer):
    sentence = remove_emoji(sentence)
    sentence = remove_links(sentence)

    str_pattern = re.compile("\r\n")
    sentence = str_pattern.sub(r'', sentence)

    sentence = re.sub('(((?![а-яА-Я ]).)+)', ' ', sentence)

    sentence = [lemmatizer.lemmatize(word) for word in nltk.word_tokenize(sentence) if word not in stop_words]
    sentence = ' '.join(sentence)
    return sentence


def get_clean_text(data):
    lemmatizer = WordNetLemmatizer()
    language = config['stopwords']
    stop_words = set(stopwords.words(language))
    comments = [preprocessing(sentence, stop_words, lemmatizer) for sentence in data]
    comments = [comm for comm in comments if len(comm) > 2]
    return comments


def vectorize_text(data, tfidf):
    mtx = tfidf.transform(data).toarray()
    mask = (np.nan_to_num(mtx) != 0).any(axis=1)
    return mtx[mask]

In [ ]:
cleaned_comments = get_clean_text(comments)
tfidf = TfidfVectorizer(**config['tf_model']).fit(cleaned_comments)
cleaned_comments[:10]

['Неплохой видос',
 'И статья',
 'На самом деле задача поиска аномалий самая простая перечисленных Если проснулись ноябре месяце рано утром г Москве улице аномалия А серьезно аномалии разыскиваются среднему отклонению среднего Причем важен модуль аномалии размер То ноября г Москвы это такая крутая аномалия',
 'якобы вычислительные мощности выросли нейронки поднялись это бред полнейший реальности нейронки поднялись открытия сверточных нейронных сетей',
 'ролик спасибо кдинственное замечание очень понятно различие классификацией кластеризацией',
 'Большое спасибо',
 'Спасибо Сделал несколько выводов студента',
 'Спасибо качественный контент',
 'Разве значение нейрона предыдущего слоя складывается значением ребра весом умножается',
 'От души ссылки рассказ']

In [ ]:
mtx = vectorize_text(cleaned_comments, tfidf)
mtx.shape, tfidf.get_feature_names_out()[:10]

((1912, 300),
 array(['автор', 'автору', 'библиотеки', 'благодарю', 'блин', 'большое',
        'буду', 'будут', 'быстро', 'ваш'], dtype=object))

In [ ]:
def get_clusters(data, count_max_clusters, random_state, affinity, silhouette_metric):
    cluster_labels = {}
    silhouette_mean = []
    for i in range(2, count_max_clusters, 1):
        clf = SpectralClustering(n_clusters=i,
                                 affinity=affinity,
                                 random_state=random_state)
        #clf = KMeans(n_clusters=n, max_iter=1000, n_init=1)
        clf.fit(data)
        labels = clf.labels_
        cluster_labels[i] = labels
        silhouette_mean.append(
            silhouette_score(data, labels, metric=silhouette_metric)
        )
    n_clusters = silhouette_mean.index(max(silhouette_mean)) + 2
    return cluster_labels[n_clusters]


def get_f1_score(y_test, y_pred, unique_cluster_labels):
    if len(unique_cluster_labels) > 2:
        return f1_score(y_test, y_pred, average='macro')
    else:
        return f1_score(y_test, y_pred)

In [ ]:
cluster_labels = get_clusters(mtx, random_state=SEED, **config['clustering'])
cluster_labels[:10]

array([12,  1,  1, 12, 12, 12, 12, 12, 12, 12], dtype=int32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mtx, cluster_labels, **config['cross_val'], random_state=SEED)

In [ ]:
clf_lr = LogisticRegression(**config['model'])

In [ ]:
%%bash
export MLFLOW_REGISTRY_URI=../mlflow

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment(config['name_experiment'])

with mlflow.start_run():
    clf_lr.fit(X_train, y_train)
    mlflow.log_param(
        'f1', get_f1_score(y_test, clf_lr.predict(X_test), set(cluster_labels))
    )
    mlflow.sklearn.log_model(
        tfidf,
        artifact_path="vector",
        registered_model_name=f"{config['model_vec']}"
    )
    mlflow.sklearn.log_model(
        clf_lr,
        artifact_path='model_lr',
        registered_model_name=f"{config['model_lr']}"
    )
    mlflow.end_run()

mlflow.get_artifact_uri()

2024/04/23 14:03:01 INFO mlflow.tracking.fluent: Experiment with name 'first' does not exist. Creating a new experiment.


2024/04/23 14:03:02 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
Successfully registered model 'vector_tfidf'.
2024/04/23 14:03:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: vector_tfidf, version 1
Created version '1' of model 'vector_tfidf'.
/home/aegon/Documents/Development/Projects/topic-sentiment-explorer/.venv/lib64/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/aegon/Documents/Development/Projects/topic-sentiment-explorer/

'/home/aegon/Documents/Development/Projects/topic-sentiment-explorer/mlflow/1/fc8352c20562487cb198f608d9a62d48/artifacts'

In [ ]:
def get_version_model(config_name, client):
    dict_push = {}
    model_versions = client.search_model_versions(f"name='{config_name}'")
    for count, value in enumerate(model_versions):
        # client.list_registered_models()):
        dict_push[count] = value
    return dict(list(dict_push.items())[-1][1])['version']

In [ ]:
client = MlflowClient()
last_version_lr = get_version_model(config['model_lr'], client)
last_version_vec = get_version_model(config['model_vec'], client)
last_version_lr, last_version_vec

('1', '1')